## Grabbing the Data
-----

### Setup:
---

In [1]:
# Import dependencies
import requests
from bs4 import BeautifulSoup
import json
from jsonpath_ng.ext import parse
import pandas as pd

In [2]:
# Create funtion for grabbing json sections
def getJSONSection(json_data, section):
    return [obj for obj in json_data if obj['name'] == section]

In [3]:
# Create function for grabbing json values
def getJSONValue(json_data, section, stat):
    section_data = getJSONSection(json_data['splits']['categories'], section)
    stat_data = getJSONSection(section_data[0]['stats'], stat)
    return stat_data[0]['displayValue']

### Grab Data:
---

In [4]:
# todo - add the rest
def getTeamUrl(team_name):
    # laliga
    if (team_name == 'Real Sociedad'):
        return 'real-sociedad'
    if (team_name == 'Villarreal'):
        return 'villarreal-cf'
    if (team_name == 'Valencia'):
        return 'valencia-cf'
    if (team_name == 'Almería'):
        return 'ud-almeria'
    

    # bundesliga
    if (team_name == 'TSG Hoffenheim'):
        return 'tsg-hoffenheim'
    if (team_name == '1. FC Union Berlin'):
        return '1-fc-union-berlin'
    if (team_name == 'Borussia Dortmund'):
        return 'borussia-dortmund'
    if (team_name == 'Bayern Munich'):
        return 'fc-bayern-muenchen'
    

    # liga mx
    if (team_name == 'Pumas UNAM'):
        return "18/pumas"
    if (team_name == 'Monterrey'):
         return "14/monterrey"
    if (team_name == 'FC Juarez'):
         return "11790/fc-juarez"
    if (team_name == 'Pachuca'):
         return "11/pachuca"
    if (team_name == 'Querétaro'):
         return "12037/queretaro"
    if (team_name == 'Mazatlán FC'):
         return "12043/mazatlan-fc"

    # ligue 1
    if (team_name == 'Metz'):
          return "fc-metz"
    if (team_name == 'Nice'):
          return "ogc-nice"
    if (team_name == 'Paris Saint-Germain'):
         return "paris-saint-germain"
    if (team_name == 'Lille'):
        return "losc-lille"
    if (team_name == 'Lyon'):
          return "olympique-lyonnais"
    if (team_name == 'Stade Rennais'):
         return "stade-rennais-fc"
    if (team_name == 'Nantes'):
          return "fc-nantes"
    if (team_name == 'Montpellier'):
         return "montpellier-herault-sc"

    # serie A
    if (team_name == 'Hellas Verona'):
         return "VERONA"
    if (team_name == 'Genoa'):
         return "GENOA"
    if (team_name == 'Sassuolo'):
         return "SASSU"
    if (team_name == 'Bologna'):
         return "BOLO"
    if (team_name == 'Fiorentina'):
         return "FIORE"
    if (team_name == 'AS Roma'):
         return "ROMA"
    if (team_name == 'AC Milan'):
         return "MILAN"
    if (team_name == 'Atalanta'):
         return "ATALAN"
    if (team_name == 'Cagliari'):
         return "CAGLIA"

TODO:

Add pagination
Don't store goalkeeper stats for non-goalkeepers

In [27]:
soccer_leagues = [
    {
        'display': 'MLS',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/usa.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'MLS',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/usa.1/seasons/2023/athletes?lang=en&region=us&limit=1000&page=2'
    },
    {
        'display': 'NWSL',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/usa.nwsl/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'Premier League',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/eng.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'LALIGA',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/esp.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'LALIGA',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/esp.1/seasons/2023/athletes?lang=en&region=us&limit=1000&page=2'
    },
    {
        'display': 'Bundesliga',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/ger.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'Bundesliga',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/ger.1/seasons/2023/athletes?lang=en&region=us&limit=1000&page=2'
    },
    {
        'display': 'Liga MX',
        'url': 'https://sports.core.api.espn.com/v2/sports/soccer/leagues/mex.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'Ligue 1',
        'url': 'https://sports.core.api.espn.com/v2/sports/soccer/leagues/fra.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'Serie A',
        'url': 'https://sports.core.api.espn.com/v2/sports/soccer/leagues/ita.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    }
]

In [28]:
selected_leagues = ['NWSL']

In [30]:
#Setup for grabbing data

#Setup lists for stats
names = []
teams = []
leagues = []
positions = []
jersey_numbers = []
goals = []
assists = []
yellow_cards = []
red_cards = []
shots = []
shots_og = []
saves = []
clean_sheets = []
goals_against = []
photo_urls = []
heights = []
weights = []
birthdates = []
nationalities = []

for league in soccer_leagues:
   if (league['display'] in selected_leagues):
     athlete_list_results_json = requests.get(league['url']).json()
    
     #Loop through players
     for athlete in range(len(athlete_list_results_json['items'])):

       athlete_results = requests.get(athlete_list_results_json['items'][athlete]['$ref'])
       athlete_results_json = athlete_results.json()
       if ('statistics' in athlete_results_json):  
         print(athlete_list_results_json['items'][athlete]['$ref'])
         team_results = requests.get(athlete_results_json['team']['$ref'])
         team_results_json = team_results.json()

         athlete_stats_results = requests.get(athlete_results_json['statistics']['$ref'])
         athlete_stats_results_json = athlete_stats_results.json()
        
         #Grab images:
         #premier league
         if league['display'] == 'Premier League':
           try:
             name_search_url = f"https://footballapi.pulselive.com/search/PremierLeague/?terms={athlete_results_json['displayName']}&type=player&size=1&start=0&fullObjectResponse=true"
             name_search_result = requests.get(name_search_url, headers = {"origin": "https://www.premierleague.com"})
             name_search_result_json = name_search_result.json()

             image_id = name_search_result_json['hits']['hit'][0]['response']['altIds']['opta']
             image_url = f"https://resources.premierleague.com/premierleague/photos/players/250x250/{image_id}.png"
           except:
             image_url = 'N/A'

         #laliga
         elif league['display'] == 'LALIGA':
           try:
             club_squad_url = f"https://www.laliga.com/en-US/clubs/{getTeamUrl(team_results_json['displayName'])}/squad"
             club_squad_results = requests.get(club_squad_url)
             club_squad_results_parsed = BeautifulSoup(club_squad_results.content.decode('utf-8'))
             club_squad_json = json.loads(club_squad_results_parsed.body.find('script', attrs={'id': '__NEXT_DATA__'}).text)
             player_element = [obj for obj in club_squad_json['props']['pageProps']['squad']['squads'] if ('shirt_number' in obj) and (str(obj['shirt_number'])) == athlete_results_json['jersey']]
             image_url =  player_element[0]['photos']['002']['512x512']
           except:
             image_url = 'N/A'

         # bundesliga
         elif league['display'] == 'Bundesliga':
           try:
             club_squad_url = f"https://www.bundesliga.com/en/bundesliga/clubs/{getTeamUrl(team_results_json['displayName'])}"
             club_squad_results = requests.get(club_squad_url)
             club_squad_results_parsed = BeautifulSoup(club_squad_results.content.decode('utf-8'))
             club_squad_json = json.loads(club_squad_results_parsed.body.find('script', attrs={'id': 'serverApp-state'}).text)
             jsonpath_expr = parse(f"$..players.*[?(@.shirtNumber={athlete_results_json['jersey']})]")
             match = [items.value['playerImages']['FACE_CIRCLE'].replace('-circle', '') for items in jsonpath_expr.find(club_squad_json)]
             image_url = match[0]
           except:
             image_url = 'N/A'

         # mls
         elif league['display'] == 'MLS':
           try:
             image_url = athlete_results_json['headshot']['href'] if 'headshot' in athlete_results_json else 'N/A'
           except:
             image_url = 'N/A'

         # nwsl
         elif league['display'] == 'NWSL':
           try:
             base_url = 'https://d2nkt8hgeld8zj.cloudfront.net'
             player_list_url = f'{base_url}/services/nwsl.ashx/players'

             player_list_results = requests.get(player_list_url)
             player_list_results_results_json = player_list_results.json()
                    
             jsonpath_expr = parse(f"$.data[?(@.shirtNumber=='{athlete_results_json['jersey']}' & @.team.title == '{team_results_json['displayName']}')]")
             match = [items.value['images']['head_shot']['url'] for items in jsonpath_expr.find(player_list_results_results_json)]
             image_url = f'{base_url}{match[0]}' if len(match) > 0 else 'N/A'
           except:
             image_url = 'N/A'

         # liga mx
         elif league['display'] == 'Liga MX':
           try:
             club_squad_url = f"https://ligamx.net/cancha/club/{getTeamUrl(team_results_json['displayName'])}"
             club_squad_results = requests.get(club_squad_url)
             club_squad_results_parsed = BeautifulSoup(club_squad_results.content.decode('utf-8'))

             match = [number.parent.parent.parent.find('img') for number in club_squad_results_parsed.find_all('strong', {'class': 'numero'}) if number.get_text() == f"#{athlete_results_json['jersey']}"]
             image_url = match[0]['src'] if len(match) > 0 else 'N/A'
           except:
             image_url = 'N/A'

         # ligue 1
         elif league['display'] == 'Ligue 1':
           try:
             base_url = "https://www.ligue1.com"
             club_squad_url = f"{base_url}/clubs/squad?id={getTeamUrl(team_results_json['displayName'])}"
             club_squad_results = requests.get(club_squad_url)
             club_squad_results_parsed = BeautifulSoup(club_squad_results.content.decode('utf-8'))

             match = [number.parent.find('img') for number in club_squad_results_parsed.find_all('div', {'class': 'SquadTeamTable-detail--number'}) if number.get_text(strip=True) == athlete_results_json['jersey']]
             image_url = f"{base_url}{match[0]['src']}" if len(match) > 0 else 'N/A'
           except:
             image_url = 'N/A'

         # serie A
         elif league['display'] == 'Serie A':
           try:
             player_results = requests.get(f"https://www.legaseriea.it/api/team/{getTeamUrl(team_results_json['displayName'])}/players")
             player_results_json = player_results.json()

             jsonpath_expr = parse(f"$..data.*[?(@.uniform_number={athlete_results_json['jersey']})]")
             match = [items.value['medium_shot'] for items in jsonpath_expr.find(player_results_json)]
             image_url = match[0]
           except:
             image_url = 'N/A'
            
         # Grab stats for current player
         cur_name = athlete_results_json['displayName']
         cur_team = team_results_json['displayName']
         cur_league = league['display']
         cur_position = athlete_results_json['position']['displayName']
         cur_jersey_number = athlete_results_json['jersey']if 'jersey' in athlete_results_json else 'N/A'
         cur_goals = getJSONValue(athlete_stats_results_json, 'offensive', 'totalGoals')
         cur_assists = getJSONValue(athlete_stats_results_json, 'offensive', 'goalAssists')
         cur_yellow_cards = getJSONValue(athlete_stats_results_json, 'general', 'yellowCards')
         cur_red_cards = getJSONValue(athlete_stats_results_json, 'general', 'redCards')
         cur_shots = getJSONValue(athlete_stats_results_json, 'offensive', 'totalShots')
         cur_shots_og = getJSONValue(athlete_stats_results_json, 'offensive', 'shotsOnTarget')
         cur_saves = getJSONValue(athlete_stats_results_json, 'goalKeeping', 'saves')
         cur_clean_sheets = getJSONValue(athlete_stats_results_json, 'goalKeeping', 'cleanSheet')
         cur_goals_against = getJSONValue(athlete_stats_results_json, 'goalKeeping', 'goalsConceded')
         cur_photo_url = image_url
         cur_height = athlete_results_json['displayHeight'] if 'displayHeight' in athlete_results_json else 'N/A'
         cur_weight = athlete_results_json['displayWeight'] if 'displayWeight' in athlete_results_json else 'N/A'
         cur_birthdate = athlete_results_json['dateOfBirth']if 'dateOfBirth' in athlete_results_json else 'N/A'
         cur_nationality = athlete_results_json['citizenship']if 'citizenship' in athlete_results_json else 'N/A'
            
         # Print summary
         print('**********')
         print(f"Name: {cur_name}")
         print(f"Team: {cur_team}")
         print(f"League: {cur_league}")
         print(f"Position: {cur_position}")
         print(f"Jersey Number: {cur_jersey_number}")
         print(f"Goals: {cur_goals}")
         print(f"Assists: {cur_assists}")
         print(f"Yellow Cards: {cur_yellow_cards}")
         print(f"Red Cards: {cur_red_cards}")
         print(f"Shots: {cur_shots}")
         print(f"Shots on Goal: {cur_shots_og}")
         print(f"Saves: {cur_saves}")
         print(f"Clean Sheets: {cur_clean_sheets}")
         print(f"Goals Against: {cur_goals_against}")
         print(f"Image URL: {cur_photo_url}")
         print(f"Height: {cur_height}")
         print(f"Weight: {cur_weight}")
         print(f"Birthdate: {cur_birthdate}")
         print(f"Nationality: {cur_nationality}")

         # Append those stats to their respective lists
         names.append(cur_name)
         teams.append(cur_team)
         leagues.append(cur_league)
         positions.append(cur_position)
         jersey_numbers.append(cur_jersey_number)
         goals.append(cur_goals)
         assists.append(cur_assists)
         yellow_cards.append(cur_yellow_cards)
         red_cards.append(cur_red_cards)
         shots.append(cur_shots)
         shots_og.append(cur_shots_og)
         saves.append(cur_saves)
         clean_sheets.append(cur_clean_sheets)
         goals_against.append(cur_goals_against)
         photo_urls.append(cur_photo_url)
         heights.append(cur_height)
         weights.append(cur_weight)
         birthdates.append(cur_birthdate)
         nationalities.append(cur_nationality) 


**********
Name: Sophia Smith
Team: Portland Thorns FC
League: NWSL
Position: Forward
Jersey Number: 9
Goals: 11
Assists: 5
Yellow Cards: 2
Red Cards: 0
Shots: 75
Shots on Goal: 39
Saves: 0
Clean Sheets: 2
Goals Against: 16
Image URL: https://d2nkt8hgeld8zj.cloudfront.net\images\2023\3\17/Sophia_Smith_001.jpeg
Height: 5' 6"
Weight: 121 lbs
Birthdate: 2000-08-10T07:00Z
Nationality: USA
**********
Name: Sophie Schmidt
Team: Houston Dash
League: NWSL
Position: Midfielder
Jersey Number: 13
Goals: 0
Assists: 0
Yellow Cards: 3
Red Cards: 0
Shots: 17
Shots on Goal: 3
Saves: 0
Clean Sheets: 8
Goals Against: 15
Image URL: https://d2nkt8hgeld8zj.cloudfront.net\images\2022\4\25/Sophie_Diana_Schmidt.png
Height: 5' 8"
Weight: 157 lbs
Birthdate: 1988-06-28T07:00Z
Nationality: Canada
**********
Name: Christine Sinclair
Team: Portland Thorns FC
League: NWSL
Position: Forward
Jersey Number: 12
Goals: 3
Assists: 1
Yellow Cards: 0
Red Cards: 0
Shots: 27
Shots on Goal: 10
Saves: 0
Clean Sheets: 1
Goals Ag

KeyError: 'team'

### Check Success and Export:
---


In [31]:
# Check pulled data length
print (len(names))

172


In [13]:
# Create athlete_statistics dictionary with lists of data
athlete_statistics = {
    "player_name" : names,
    "team" : teams,
    "league" : leagues,
    "position" : positions,
    "jersey_number" : jersey_numbers,
    "goals" : goals,
    "assists" : assists,
    "yellow_cards" : yellow_cards,
    "red_cards" : red_cards,
    "shots" : shots,
    "shots_on_goal" : shots_og,
    "saves" : saves,
    "clean_sheets" : clean_sheets,
    "goals_against" : goals_against,
    "photo_url" : photo_urls,
    "height" : heights,
    "weight" : weights,
    "birthdate" : birthdates,
    "nationality" : nationalities
}

In [14]:
# View uncleaned dataframe
athelete_stats_df = pd.DataFrame(athlete_statistics)
athelete_stats_df

,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,clean_sheets,goals_against,photo_url,height,weight,birthdate,nationality
0,Sophia Smith,Portland Thorns FC,nwsl,Forward,9,11,5,2,0,75,39,0,2,16,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 6""",121 lbs,2000-08-10T07:00Z,USA
1,Sophie Schmidt,Houston Dash,nwsl,Midfielder,13,0,0,3,0,17,3,0,8,15,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 8""",157 lbs,1988-06-28T07:00Z,Canada
2,Christine Sinclair,Portland Thorns FC,nwsl,Forward,12,3,1,0,0,27,10,0,1,12,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 9""",148 lbs,1983-06-12T07:00Z,Canada
3,Abby Erceg,Racing Louisville FC,nwsl,Defender,20,2,0,1,0,15,6,0,6,22,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 10""",150 lbs,1989-11-20T08:00Z,New Zealand
4,Nadia Nadim,Racing Louisville FC,nwsl,Forward,10,1,0,0,0,14,5,0,0,5,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 9""",N/A,1988-01-02T08:00Z,Denmark
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,Chloe Ricketts,Washington Spirit,nwsl,Midfielder,39,0,0,1,0,3,1,0,0,3,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 1""",N/A,2007-05-23T07:00Z,USA
320,Melanie Barcenas,San Diego Wave FC,nwsl,Forward,25,0,1,0,0,6,4,0,0,2,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 4""",N/A,2007-10-30T07:00Z,USA
321,Riley Parker,Racing Louisville FC,nwsl,Midfielder,N/A,0,0,0,0,0,0,0,0,0,N/A,N/A,N/A,2000-03-31T08:00Z,USA
322,Brianna Martínez,Orlando Pride,nwsl,Defender,32,0,0,0,0,0,0,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,N/A,N/A,2000-04-22T07:00Z,USA


In [15]:
# Export unclean data to CSV
athelete_stats_df.to_csv ("initial_unclean_csv/US_unclean_data.csv", encoding='utf8', index=False, header=True)
